# Testing GISSR-style inundation on ideal topo

Imports and setups

In [1]:
import pandas as pd
import numpy as np

import params_ideal as params

from fun_ideal import calc_flood_height, generate_groups, calc_group_vol, calc_group_h
from ideal_damage import Damage
from topo_ideal import Topo

In [2]:
topo_file = "Input/ideal_topo_gp_points_1.csv"
storm_file = "SurgeData/surge_w.csv"
div_data_file = "Input/ideal_div_data.csv"
time_file = "SurgeData/time_w.csv"
damage_table_file = "Input/damage_table.csv"

In [3]:
surge = pd.read_csv(storm_file).values
surge_time = pd.read_csv(time_file).values
damage_table = pd.read_csv(damage_table_file)


Topo = Topo(topo_file=topo_file)
Damage = Damage(damage_table_file)

In [4]:
# generate groups
#groups = generate_groups(Topo.div_data)

segment_l = 100
wall_height = np.zeros(Topo.shore_height.size)


# Calculate indundation with varying sized walls
Equal height wall for all divs in divs_allocate

In [14]:
divs_allocate = [1]
groups = generate_groups(Topo.div_data)

positions = []
for i, p in enumerate(wall_height):
    if Topo.all_divs[i] in divs_allocate:
        positions.append(i)
        
h_start = 0
h_end = 2
h_num = 80

points = []
wall_height[positions] = h_start

for h in np.linspace(h_start, h_end, num=h_num):
    wall_height[:] = 0
    wall_height[positions] = h

    height_div, volume_div = calc_flood_height(Topo, surge, surge_time, wall_height, positions)

    # calculate group volumes
    volume_grouped = calc_group_vol(groups, volume_div)

    # get height in each group
    surge_peak = np.amax(surge, axis=1)
    height_grouped = calc_group_h(Topo, groups, volume_grouped, surge_peak)

    #Damage
    dmg = Damage.calc_damage(height_grouped)
    cost_dmg = np.sum(dmg)
    cost_wall = np.sum(49212 * wall_height * segment_l)
    cost_tot = cost_dmg + cost_wall

    points.append([h, cost_wall, cost_dmg, cost_tot, volume_div[4,1]])
    print(height_div[4,1])

    
df = pd.DataFrame(points)
df.to_csv("Output/single_section_varying_wall.csv")
#print(df)

IndexError: index 4 is out of bounds for axis 0 with size 1

# Inundation with varying travel distance for redistribution

In [5]:
divs_allocate = list(range(18))

positions = []
for i, p in enumerate(wall_height):
    if Topo.all_divs[i] in divs_allocate:
        positions.append(i)
        
h_start = 0
h_end = 0
h_num = 1

points = []
wall_height[positions] = h_start

data = []
for r in range(18):
    params.travel_dist = r
    groups = generate_groups(Topo.div_data)

    for h in np.linspace(h_start, h_end, num=h_num):
        wall_height[:] = 0
        wall_height[positions] = h

        height_div, volume_div = calc_flood_height(Topo, surge, surge_time, wall_height, positions)

        # calculate group volumes
        volume_grouped = calc_group_vol(groups, volume_div)

        # get height in each group
        surge_peak = np.amax(surge, axis=1)
        height_grouped = calc_group_h(Topo, groups, volume_grouped, surge_peak)
        
        #Damage
        dmg = Damage.calc_damage(height_grouped)
        cost_dmg = np.sum(dmg)
        cost_wall = np.sum(49212 * wall_height * segment_l)
        cost_tot = cost_dmg + cost_wall

        points.append([h, cost_wall, cost_dmg, cost_tot])

        data.append(height_grouped[2])
        #print(np.max(np.max(height_grouped, axis=1) - np.min(height_grouped, axis=1)))

df = pd.DataFrame(data)
df.to_csv("Output/ideal_test_travel.csv")

# PLOT


In [6]:
print(df)

          0         1         2         3         4         5         6   \
0   0.233891  0.263045  0.233891  0.270074  0.270074  0.270074  0.190971   
1   0.248895  0.243996  0.256149  0.258576  0.270074  0.246543  0.190971   
2   0.243996  0.250770  0.254748  0.261808  0.248996  0.225960  0.190971   
3   0.250770  0.254748  0.257366  0.248967  0.232744  0.210440  0.190971   
4   0.254748  0.257366  0.248967  0.232887  0.219568  0.205261  0.190971   
5   0.257366  0.248967  0.232887  0.219568  0.208301  0.202737  0.190077   
6   0.248967  0.232887  0.219568  0.208301  0.202737  0.194106  0.186491   
7   0.232887  0.219568  0.208301  0.202737  0.194106  0.186491  0.179708   
8   0.219568  0.208301  0.202737  0.194106  0.186491  0.179708  0.173614   
9   0.208301  0.202737  0.194106  0.186491  0.179708  0.173614  0.168101   
10  0.202737  0.194106  0.186491  0.179708  0.173614  0.168101  0.172666   
11  0.194106  0.186491  0.179708  0.173614  0.168101  0.172666  0.172384   
12  0.186491

We can see below that as the travel distance is increased the water level becomes more and more homogenous

In [7]:
import matplotlib.pyplot as plt

divs = list(range(18))
for div in divs:
    plt.plot(divs, data[div], label=div)
#plt.legend()
plt.show()